# 네이버 블로그 크롤링 Selenium + Beautifulsoup

In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

from pathlib import Path
import time
import os

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:02<00:00, 2.99MB/s]


In [3]:
str(Path('crwl_result_E/'))+'ENFP.csv'

'crwl_result_EENFP.csv'

In [4]:
# 검색 리스트
mbti_e = ['ENFP', 'ENFJ', 'ENTP', 'ENTJ',
          'ESFP', 'ESFJ', 'ESTP', 'ESTJ']

In [5]:
# 검색 함수
def Crawl(k):
    # k <- keyword
    k = k.replace(' ','+')
    browser = webdriver.Chrome(service=service,options=chrome_options)
    browser.maximize_window()
    url = f"https://search.naver.com/search.naver?query={k}&nso=&where=blog&sm=tab_opt"
    browser.get(url)
    browser.implicitly_wait(2)

    crwl_blog = []
    # 스크롤 횟수 
    for i in range(4):
        browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    links = browser.find_elements(By.CSS_SELECTOR,"div.total_area > a")
    blog_url = []
    for link in links:
        blog_url.append(link.get_attribute('href'))

    for i in range(len(blog_url)):
        blog_text = ''
        browser.get(blog_url[i])
        browser.implicitly_wait(2.5)
        browser.switch_to.frame("mainFrame")
        soup = BeautifulSoup(browser.page_source,'html.parser')
        text=soup.select("div.se-module-text")
        str_=''
        for t in text:
            blog_text += t.text
        crwl_blog.append(blog_text)
        if i==10: break

    print(crwl_blog)
    browser.close()
    return crwl_blog

In [6]:
def ToCsv(k, result):
    ''' 검색 결과 CSV 변환 '''
    save_name = k + '.csv'
    pd.DataFrame(result).to_csv(save_name, encoding='UTF-8')

In [6]:
for k in mbti_e:
    key = f"{k} 특징"
    crwl_result = Crawl(key)
    ToCsv(k, crwl_result)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C4A813+48355]
	(No symbol) [0x00BDC4B1]
	(No symbol) [0x00AE5358]
	(No symbol) [0x00ACD293]
	(No symbol) [0x00B2E37B]
	(No symbol) [0x00B3C473]
	(No symbol) [0x00B2A536]
	(No symbol) [0x00B082DC]
	(No symbol) [0x00B093DD]
	GetHandleVerifier [0x00EAAABD+2539405]
	GetHandleVerifier [0x00EEA78F+2800735]
	GetHandleVerifier [0x00EE456C+2775612]
	GetHandleVerifier [0x00CD51E0+616112]
	(No symbol) [0x00BE5F8C]
	(No symbol) [0x00BE2328]
	(No symbol) [0x00BE240B]
	(No symbol) [0x00BD4FF7]
	BaseThreadInitThunk [0x76E500C9+25]
	RtlGetAppContainerNamedObjectPath [0x77377B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77377B1E+238]


In [91]:
keyword_input = input('검색 단어 입력 : ')
keyword_input = keyword_input.replace(' ','+')

browser = webdriver.Chrome(service=service,options=chrome_options)
browser.maximize_window()
url = f"https://search.naver.com/search.naver?query={keyword_input}&nso=&where=blog&sm=tab_opt"
browser.get(url)
browser.implicitly_wait(2)

titles = []
crwl_blog = []
for i in range(4):
    browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
    time.sleep(1)
    browser.execute_script("return window.scrollY")
links = browser.find_elements(By.CSS_SELECTOR,"div.total_area > a")
blog_url = []
for link in links:
    blog_url.append(link.get_attribute('href'))

for i in range(len(blog_url)):
    blog_text = ''
    browser.get(blog_url[i])
    browser.implicitly_wait(2.5)
    browser.switch_to.frame("mainFrame")
    soup = BeautifulSoup(browser.page_source,'html.parser')
    
    # title 긁어 오기
    title = soup.select("div.pcol1")
    for element in soup.select("div.pcol1"):
        text = element.get_text(strip=True)
        titles.append(text)

    # 본문 긁어 오기
    text = soup.select("div.se-main-container")
    #text = soup.select('div.se-component-content') # 이웃추가까지 긁힘
    for t in text:
        blog_text += t.text
    blog_text = re.sub('\n+', ' ', blog_text)
    blog_text = re.sub('\u200b+', ' ', blog_text)
    crwl_blog.append(blog_text)
    if i==10: break

print(titles)
print(crwl_blog)




# time.sleep(2)
# browser.find_elements(By.CSS_SELECTOR,"div.tlink_nosub").send_keys(Keys.END)

# iframe 요소 불러와지는지 확인
# a=browser.find_elements(By.CSS_SELECTOR,"#mainFrame")
# print(a)

# iframe으로 전환
# browser.switch_to.frame("mainFrame")
# z = browser.find_elements(By.CSS_SELECTOR, "div.tlink_nosub")

['enfp 특징 성격과 여행 스타일을 소개합니다.', 'enfp 연애 특징 팩폭 금사빠 금사식 바람둥이 오해 풀어준다', 'MBTI 영어 뜻 ENFP 여자 특징 연애 단점 ENFP-T', 'isfp 특징 & enfp 특징 정리! 장단점부터 성격 궁합까지❣', 'enfp 특징 모음! 연애, 팩폭, 빙고까지', 'ENFP 연애,궁합,특징,이상형 (팩폭정리)', 'ENFP 장단점 특징 열정 가득한 그들 이해하기', 'ENFP 특징 연애 궁합 유형 성별로 알아보기!', 'isfp 특징 정리! (enfp 포함) 성격유형검사 한 눈에 보기!', 'MBTI 성격 테스트, 정식 엠비티아이 검사와 ENFP 특징', 'ENFP 특징 성격 팩폭 장단점 까지!']
['enfp 특징 성격과 여행 스타일을 소개합니다.상호 친밀감을 위해서 말은 편해게 해볼게요.우선, enfp는 우리나라에서 mbti 2위에 랭킹 될 정도로 많은 사람들이 해당되는 타입이고요, 도전적이지만 장기적으로 꾸준하게 하는 거 안 좋아하고 즉흥적이면서 열정적인 활동가 스타일입니다. 상상력창의력리더십정의 실현도전 정신성격이 예민함.계획이 뭐예요?집중력이 낮음열정적  도전적인 거 좋아합니다.남들이 하지 않거나, 하고 싶어 하는데 쉽사리 못하는 거 있죠?예를 들어, 스카이다이빙이라 패러글라이딩, 스쿠버다이빙 같은 액티비티 잘 합니다. 적극적으로 생활하고 열정적이라서 학창 시절에는 학급 임원 많이 도전했을 거고요, 자부심이 느낄 수 있는 일을 좋아하고 자부심을 느낄 수 있는 일을 하게 되면 힘든 줄 모르고 열심히 합니다. 저예요혼자 잘 놀아요.어디서 본 거는 많아서, 감성적인 시간 좋아하고요 여행 혼자 가서도 잘 놀고 옵니다. 물론, 여러 사람들과 춤추고 파티하면서 놀고 오는 게 아니고 혼자서 여기저기 잘 다니고 잘 찾아다니면서 먹을 거 먹고 잘 다니는 스타일입니다. 계획이 없어요.시험공부 하면서 미리미리 계획을 하기는 하는데요, 나름 전략가라서 미리미리 어떻게 공부를 하고 요약 페이지를 만들어 체계적으로 공부하는 스타일

In [99]:
'blog.naver.com' in blog_url[0]

True

In [92]:
crwl_blog[0]

'enfp 특징 성격과 여행 스타일을 소개합니다.상호 친밀감을 위해서 말은 편해게 해볼게요.우선, enfp는 우리나라에서 mbti 2위에 랭킹 될 정도로 많은 사람들이 해당되는 타입이고요, 도전적이지만 장기적으로 꾸준하게 하는 거 안 좋아하고 즉흥적이면서 열정적인 활동가 스타일입니다. 상상력창의력리더십정의 실현도전 정신성격이 예민함.계획이 뭐예요?집중력이 낮음열정적  도전적인 거 좋아합니다.남들이 하지 않거나, 하고 싶어 하는데 쉽사리 못하는 거 있죠?예를 들어, 스카이다이빙이라 패러글라이딩, 스쿠버다이빙 같은 액티비티 잘 합니다. 적극적으로 생활하고 열정적이라서 학창 시절에는 학급 임원 많이 도전했을 거고요, 자부심이 느낄 수 있는 일을 좋아하고 자부심을 느낄 수 있는 일을 하게 되면 힘든 줄 모르고 열심히 합니다. 저예요혼자 잘 놀아요.어디서 본 거는 많아서, 감성적인 시간 좋아하고요 여행 혼자 가서도 잘 놀고 옵니다. 물론, 여러 사람들과 춤추고 파티하면서 놀고 오는 게 아니고 혼자서 여기저기 잘 다니고 잘 찾아다니면서 먹을 거 먹고 잘 다니는 스타일입니다. 계획이 없어요.시험공부 하면서 미리미리 계획을 하기는 하는데요, 나름 전략가라서 미리미리 어떻게 공부를 하고 요약 페이지를 만들어 체계적으로 공부하는 스타일입니다. 그런 면에서는 철저한 준비를 하는 편인데, 그것 외에 평상시 여행이라던가 생활에 있어서는 이렇다 하게 계획 없는 즉흥적인 걸 좋아합니다. P거든요.예를 들어, P와 J의 여행 스케줄 표를 비교해 볼게요. J의 여행 계획표일정1일차2일차3일차J 스치는 바람에~09시공항 도착오페라하우스노스 시드니 이동10시숙소 이동천문대파크 도착11시식당 출발루나파크마켓 도착12시시내 스테이크 맛집 도착페리로 서큘러 키 이동쇼핑몰에서 점심  P의 여행 계획표일정1일차2일차3일차이마저도 지켜지지 않음.09시공항 도착시내10시숙소 이동11시시내12시 계획표 자체가 없어요.. 저도 enfp입니다.이번에 시드니 여행을 하였는데요, 제가 enfp라서 공항에서 호텔까지

In [95]:
pd.DataFrame(zip(titles, crwl_blog), columns = ['title', 'content'])

,title,content
0,enfp 특징 성격과 여행 스타일을 소개합니다.,enfp 특징 성격과 여행 스타일을 소개합니다.상호 친밀감을 위해서 말은 편해게 해...
1,enfp 연애 특징 팩폭 금사빠 금사식 바람둥이 오해 풀어준다,"enfp는 사랑이 넘치는 mbti 유형이에요처음보는 사람들과 쉽게 친해지고,i 유..."
2,MBTI 영어 뜻 ENFP 여자 특징 연애 단점 ENFP-T,MBTI 영어 뜻 ENFP 여자 특징 연애 단점 ENFP-T 오늘 이야기 할 주제는...
3,isfp 특징 & enfp 특징 정리! 장단점부터 성격 궁합까지❣,지난번 mbti 포스팅에서 유형별 성격과특징 대해 간단히 정리하였습니다 (⑅ ᵕᴗᵕ...
4,"enfp 특징 모음! 연애, 팩폭, 빙고까지",안녕하세요! 메이입니다. 이번에 저와 살펴볼 mbti는 바로 인싸형으로 알려진 엔프...
5,"ENFP 연애,궁합,특징,이상형 (팩폭정리)",안녕하세요 여러분 오늘은 ENFP 연애에대해 알아볼까해요성격유형은 재기발랄한 활동가...
6,ENFP 장단점 특징 열정 가득한 그들 이해하기,정말 열정 가득한 MBTI 유형은 누구일까요?각자 다들 열정이 있고 정도의 차이는 ...
7,ENFP 특징 연애 궁합 유형 성별로 알아보기!,안녕하세요 해커스공무원입니다.오늘은 ENFP에 대한 특징과 연애 궁합에대해서 알아 ...
8,isfp 특징 정리! (enfp 포함) 성격유형검사 한 눈에 보기!,요즘 MBTI 검사 정말 많이들 하죠!!??저도 원래 요런거에 관심이 없어서 안하...
9,"MBTI 성격 테스트, 정식 엠비티아이 검사와 ENFP 특징","눈오리!이 글을 쓰고 있는 지금 이 시간에도,밖에선 함박눈이 펑펑 내리고 있습니다...."


In [85]:
import re
re.sub('\n+', '', crwl_blog[0])

'enfp 특징 성격과 여행 스타일을 소개합니다.상호 친밀감을 위해서 말은 편해게 해볼게요.우선, enfp는 우리나라에서 mbti 2위에 랭킹 될 정도로 많은 사람들이 해당되는 타입이고요, 도전적이지만 장기적으로 꾸준하게 하는 거 안 좋아하고 즉흥적이면서 열정적인 활동가 스타일입니다.\u200b\u200b 상상력창의력리더십정의 실현도전 정신성격이 예민함.계획이 뭐예요?집중력이 낮음열정적 \u200b\u200b 도전적인 거 좋아합니다.남들이 하지 않거나, 하고 싶어 하는데 쉽사리 못하는 거 있죠?예를 들어, 스카이다이빙이라 패러글라이딩, 스쿠버다이빙 같은 액티비티 잘 합니다. 적극적으로 생활하고 열정적이라서 학창 시절에는 학급 임원 많이 도전했을 거고요, 자부심이 느낄 수 있는 일을 좋아하고 자부심을 느낄 수 있는 일을 하게 되면 힘든 줄 모르고 열심히 합니다.\u200b\u200b 저예요혼자 잘 놀아요.어디서 본 거는 많아서, 감성적인 시간 좋아하고요 여행 혼자 가서도 잘 놀고 옵니다. 물론, 여러 사람들과 춤추고 파티하면서 놀고 오는 게 아니고 혼자서 여기저기 잘 다니고 잘 찾아다니면서 먹을 거 먹고 잘 다니는 스타일입니다.\u200b\u200b 계획이 없어요.시험공부 하면서 미리미리 계획을 하기는 하는데요, 나름 전략가라서 미리미리 어떻게 공부를 하고 요약 페이지를 만들어 체계적으로 공부하는 스타일입니다. 그런 면에서는 철저한 준비를 하는 편인데, 그것 외에 평상시 여행이라던가 생활에 있어서는 이렇다 하게 계획 없는 즉흥적인 걸 좋아합니다. P거든요.\u200b\u200b예를 들어, P와 J의 여행 스케줄 표를 비교해 볼게요.\u200b J의 여행 계획표일정1일차2일차3일차J 스치는 바람에~09시공항 도착오페라하우스노스 시드니 이동10시숙소 이동천문대파크 도착11시식당 출발루나파크마켓 도착12시시내 스테이크 맛집 도착페리로 서큘러 키 이동쇼핑몰에서 점심 \u200b\u200b\u200b P의 여행 계획표일정1일차2일차3일차이마저도 지켜지지 않음.09시공항 

In [75]:
len(blog_url)

150

In [65]:
'\\n'

'\\n'

In [71]:
crwl_blog[0]

'\n\n\n\n스크랩\n\n\n\n\nenfp 특징 성격과 여행 스타일을 소개합니다. \n\n\n\n\n\n프렌시\n\n ・ \n2023. 5. 31. 1:25\n\n\nURL 복사\n 이웃추가\n\n본문 기타 기능\n\n\n                   공유하기\n                \n\n신고하기\n\n\n\n\n\n\n\n\n\nenfp 특징 성격과 여행 스타일을 소개합니다.\n\n\n\n\n\n상호 친밀감을 위해서 말은 편해게 해볼게요.우선, enfp는 우리나라에서 mbti 2위에 랭킹 될 정도로 많은 사람들이 해당되는 타입이고요, 도전적이지만 장기적으로 꾸준하게 하는 거 안 좋아하고 즉흥적이면서 열정적인 활동가 스타일입니다.\u200b\u200b\n\n\n\n\n\n\n\n상상력\n\n창의력\n\n리더십\n\n정의 실현\n\n도전 정신\n\n성격이 예민함.\n\n계획이 뭐예요?\n\n집중력이 낮음\n\n열정적\n\n\n\n\n\n\n\n\u200b\u200b\n\n\n\n\n\n\n\n\n\n\n\n\n\n도전적인 거 좋아합니다.\n\n\n\n\n\n남들이 하지 않거나, 하고 싶어 하는데 쉽사리 못하는 거 있죠?예를 들어, 스카이다이빙이라 패러글라이딩, 스쿠버다이빙 같은 액티비티 잘 합니다. 적극적으로 생활하고 열정적이라서 학창 시절에는 학급 임원 많이 도전했을 거고요, 자부심이 느낄 수 있는 일을 좋아하고 자부심을 느낄 수 있는 일을 하게 되면 힘든 줄 모르고 열심히 합니다.\u200b\u200b\n\n\n\n\n\n\n\n\n\n저예요\n\n\n\n\n혼자 잘 놀아요.\n\n\n\n\n\n어디서 본 거는 많아서, 감성적인 시간 좋아하고요 여행 혼자 가서도 잘 놀고 옵니다. 물론, 여러 사람들과 춤추고 파티하면서 놀고 오는 게 아니고 혼자서 여기저기 잘 다니고 잘 찾아다니면서 먹을 거 먹고 잘 다니는 스타일입니다.\u200b\u200b\n\n\n\n\n\n\n\n\n\n\n\n\n\n계획이 없어요.\n\n\n\n\n\n시험공부 하면서 

In [72]:
soup.select('se-component se-text se-l-default')

[]

In [62]:
text = soup.select('div.se-component-content')
for t in text:
    blog_text += t.text
crwl_blog.append(blog_text)
print(crwl_blog)

['\n\n\n\n\n\u200b\u200b\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b며칠전 드라마를 봤다드라마에서 한 커플이 있는데 여자가 외국에 인턴쉽이 되서 외국을 떠나야하는 상황이 생겼다. 남자는 그렇게 가면 헤어질거같으니 가지말라고하는 상황이었다. 그래서 순간 궁금해져서 신랑한테 물었다\u200b\u200b나-오빠는 내가 만약 워킹홀리데이를 간다고 하면 어떨거가테?오빠-ㅋㅋㅋㅋㅋㅋ 말이되는소릴해야지 자긴 귀차나서 못해ㅋㅋㅋㅋㅋㅋ\u200b나- 뭐!? 아니 그니까 만약이라고 말한거자나! 만약 내가 간다고 하면~~~오빠- 아니 그니까 말이되는소리여야지 그겤ㅋㅋㅋㅋ 워홀이 비자 받기도 까다롭고 얼마나 절차가 많은줄알아? 자기가 그걸 한다고?ㅋㅋㅋㅋㅋㅋㅋ귀차나서 못해 못가 ㅋㅋㅋㅋㅋㅋ\u200b나-그니까 만약이라자나!! 내가 외국에 멀리 떨어져서 살수있는 상황이 되면 어쩔거냐고~오빠-ㅋㅋㅋㅋ 아니 내가 주재원에 가는게 빠르겠다 무슨 실현가능성이 35%라도 있어야지 뭘 생각이라는걸 해보지 말도안되는걸 어떻게 자꾸 물어봐 \u200b로 끝난 대화 \u200b\u200b\u200b\u200b순간 낮에 인터넷에서 봤던 S와N 차이점이 생각이 났다민간인사찰 당한듯 너무 정확히 우리 부부 대화였던ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\u200bS는 sensing 감각형이고 N은 iNtuition 직관형으로F와 T 보다 더 극복하기가 어렵다는 S/N대화 장벽이 있던걸 결혼 4년차에 접어들어서야 깨닫게 되었다\u200b\u200b\u200b\u200bS는 실현가능성이 없는 만약을 싫어하는듯  하다. 만약이라는 말이 붙는것도 신랑의 말대로 실현가능성이 어느정도 있어야지 가정할수있고 대화를 하는데 아예 말도 안되는건 대화가 불가능이었던거다 \u200b그동안 S/N의 차이가 사실 크게 와닿지 않아서 잘 모르고 있었는데 이렇게 차이가 많이 났던걸 내가 인지하지 못하고 지냈던거라니! s/n은 예시를 봐도 이해가 잘 안되고 납득이 안되서 잘 모르겠던 것

In [33]:
crwl_blog

['\n\n\n\n\n\u200b\u200b\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b며칠전 드라마를 봤다드라마에서 한 커플이 있는데 여자가 외국에 인턴쉽이 되서 외국을 떠나야하는 상황이 생겼다. 남자는 그렇게 가면 헤어질거같으니 가지말라고하는 상황이었다. 그래서 순간 궁금해져서 신랑한테 물었다\u200b\u200b나-오빠는 내가 만약 워킹홀리데이를 간다고 하면 어떨거가테?오빠-ㅋㅋㅋㅋㅋㅋ 말이되는소릴해야지 자긴 귀차나서 못해ㅋㅋㅋㅋㅋㅋ\u200b나- 뭐!? 아니 그니까 만약이라고 말한거자나! 만약 내가 간다고 하면~~~오빠- 아니 그니까 말이되는소리여야지 그겤ㅋㅋㅋㅋ 워홀이 비자 받기도 까다롭고 얼마나 절차가 많은줄알아? 자기가 그걸 한다고?ㅋㅋㅋㅋㅋㅋㅋ귀차나서 못해 못가 ㅋㅋㅋㅋㅋㅋ\u200b나-그니까 만약이라자나!! 내가 외국에 멀리 떨어져서 살수있는 상황이 되면 어쩔거냐고~오빠-ㅋㅋㅋㅋ 아니 내가 주재원에 가는게 빠르겠다 무슨 실현가능성이 35%라도 있어야지 뭘 생각이라는걸 해보지 말도안되는걸 어떻게 자꾸 물어봐 \u200b로 끝난 대화 \u200b\u200b\u200b\u200b순간 낮에 인터넷에서 봤던 S와N 차이점이 생각이 났다민간인사찰 당한듯 너무 정확히 우리 부부 대화였던ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\u200bS는 sensing 감각형이고 N은 iNtuition 직관형으로F와 T 보다 더 극복하기가 어렵다는 S/N대화 장벽이 있던걸 결혼 4년차에 접어들어서야 깨닫게 되었다\u200b\u200b\u200b\u200bS는 실현가능성이 없는 만약을 싫어하는듯  하다. 만약이라는 말이 붙는것도 신랑의 말대로 실현가능성이 어느정도 있어야지 가정할수있고 대화를 하는데 아예 말도 안되는건 대화가 불가능이었던거다 \u200b그동안 S/N의 차이가 사실 크게 와닿지 않아서 잘 모르고 있었는데 이렇게 차이가 많이 났던걸 내가 인지하지 못하고 지냈던거라니! s/n은 예시를 봐도 이해가 잘 안되고 납득이 안되서 잘 모르겠던 것

In [9]:
blog_text = ''
browser.get(blog_url[i])
browser.implicitly_wait(2.5)
browser.switch_to.frame("mainFrame")
soup = BeautifulSoup(browser.page_source,'html.parser')

#title 긁어 오기
title = soup.select("div.pcol1")
text=soup.select("div.se-main-container")
#str_=''

for t in text:
    blog_text += t.text
    crwl_blog.append(blog_text)
    if i==10: break

print(crwl_blog)


['\n\n\n\n\n\u200b\u200b\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b며칠전 드라마를 봤다드라마에서 한 커플이 있는데 여자가 외국에 인턴쉽이 되서 외국을 떠나야하는 상황이 생겼다. 남자는 그렇게 가면 헤어질거같으니 가지말라고하는 상황이었다. 그래서 순간 궁금해져서 신랑한테 물었다\u200b\u200b나-오빠는 내가 만약 워킹홀리데이를 간다고 하면 어떨거가테?오빠-ㅋㅋㅋㅋㅋㅋ 말이되는소릴해야지 자긴 귀차나서 못해ㅋㅋㅋㅋㅋㅋ\u200b나- 뭐!? 아니 그니까 만약이라고 말한거자나! 만약 내가 간다고 하면~~~오빠- 아니 그니까 말이되는소리여야지 그겤ㅋㅋㅋㅋ 워홀이 비자 받기도 까다롭고 얼마나 절차가 많은줄알아? 자기가 그걸 한다고?ㅋㅋㅋㅋㅋㅋㅋ귀차나서 못해 못가 ㅋㅋㅋㅋㅋㅋ\u200b나-그니까 만약이라자나!! 내가 외국에 멀리 떨어져서 살수있는 상황이 되면 어쩔거냐고~오빠-ㅋㅋㅋㅋ 아니 내가 주재원에 가는게 빠르겠다 무슨 실현가능성이 35%라도 있어야지 뭘 생각이라는걸 해보지 말도안되는걸 어떻게 자꾸 물어봐 \u200b로 끝난 대화 \u200b\u200b\u200b\u200b순간 낮에 인터넷에서 봤던 S와N 차이점이 생각이 났다민간인사찰 당한듯 너무 정확히 우리 부부 대화였던ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\u200bS는 sensing 감각형이고 N은 iNtuition 직관형으로F와 T 보다 더 극복하기가 어렵다는 S/N대화 장벽이 있던걸 결혼 4년차에 접어들어서야 깨닫게 되었다\u200b\u200b\u200b\u200bS는 실현가능성이 없는 만약을 싫어하는듯  하다. 만약이라는 말이 붙는것도 신랑의 말대로 실현가능성이 어느정도 있어야지 가정할수있고 대화를 하는데 아예 말도 안되는건 대화가 불가능이었던거다 \u200b그동안 S/N의 차이가 사실 크게 와닿지 않아서 잘 모르고 있었는데 이렇게 차이가 많이 났던걸 내가 인지하지 못하고 지냈던거라니! s/n은 예시를 봐도 이해가 잘 안되고 납득이 안되서 잘 모르겠던 것

In [10]:
blog_title = ''
titles = ''

for t in title:
    blog_title += t.title
    titles.append(blog_title)
print(titles)

TypeError: can only concatenate str (not "NoneType") to str

In [56]:
soup.select("div.pcol1")

[<div class="pcol1">
 <!-- -->
 <div class="se-module se-module-text se-title-text">
 <p class="se-text-paragraph se-text-paragraph-align-" id="SE-1fe6a678-7c80-4318-869d-7a761dee09ad" style=""><span class="se-fs- se-ff-" id="SE-091c2e65-9612-4db1-b405-b2138bd0f4f9" style=""><!-- -->ENFP ISTP 남자 여자 궁합 특징<!-- --></span></p> </div>
 <!-- -->
 </div>]

In [57]:
#GPT
for element in soup.select("div.pcol1"):
    text = element.get_text(strip=True)
    print(text)

ENFP ISTP 남자 여자 궁합 특징


In [40]:
#지성님
soup.select_one('p.se-text-paragraph.se-text-paragraph-align-').text

'ENFP ISTP 남자 여자 궁합 특징'

In [26]:
test.

[<div class="pcol1">
 <!-- -->
 <div class="se-module se-module-text se-title-text">
 <p class="se-text-paragraph se-text-paragraph-align-" id="SE-1fe6a678-7c80-4318-869d-7a761dee09ad" style=""><span class="se-fs- se-ff-" id="SE-091c2e65-9612-4db1-b405-b2138bd0f4f9" style=""><!-- -->ENFP ISTP 남자 여자 궁합 특징<!-- --></span></p> </div>
 <!-- -->
 </div>]

In [ ]:
# 결과 저장
save_path = Path('crwl_result_E/')
save_csv = save_path + k + '.csv'
pd.DataFrame(crwl_blog).to_csv(save_csv)
pd.DataFrame(crwl_blog).to_csv('C:\python\team_project\finalProject\crwl_result_E/ENFP.csv')

In [ ]:
P